In [1]:
import tensorflow as tf
print(tf.config.list_physical_devices())

import gc
class FreeMemory(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f'epoch {epoch} ended, info: {logs}')
        tf.keras.backend.clear_session()
        gc.collect()

2023-10-14 15:04:32.772721: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-10-14 15:04:35.527432: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-14 15:04:35.545563: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-14 15:04:35.545811: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
import numpy as np
raw_data = np.load('/host/code/x_y_phi.npy')
data_x = [x + 1.j*y for x, y in raw_data[:,:2]]
data_y = raw_data[:,-1]

In [5]:
import cvnn.layers as complex_layers
from cvnn.losses import ComplexMeanSquareError
from tensorflow.keras.optimizers import SGD, Adam

model = tf.keras.models.Sequential()
model.add(complex_layers.ComplexInput(input_shape=(1,)))
model.add(complex_layers.ComplexDense(units=32, activation='pol_tanh'))
model.add(complex_layers.ComplexDense(units=1, activation='sigmoid_real'))
model.compile(loss=ComplexMeanSquareError(), optimizer=SGD())

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 complex_dense (ComplexDens  (None, 32)                128       
 e)                                                              
                                                                 
 complex_dense_1 (ComplexDe  (None, 1)                 66        
 nse)                                                            
                                                                 
Total params: 194 (776.00 Byte)
Trainable params: 194 (776.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [6]:
history = model.fit(np.array(data_x), data_y, epochs=10, validation_split=0.2, verbose=0, callbacks=[FreeMemory()])

epoch 0 ended, info: {'loss': 2.2740001678466797, 'val_loss': 2.3446130752563477}
epoch 1 ended, info: {'loss': 2.240057945251465, 'val_loss': 2.3337345123291016}
epoch 2 ended, info: {'loss': 2.230614185333252, 'val_loss': 2.3264212608337402}
epoch 3 ended, info: {'loss': 2.2250678539276123, 'val_loss': 2.3223164081573486}
epoch 4 ended, info: {'loss': 2.2206809520721436, 'val_loss': 2.3212015628814697}
epoch 5 ended, info: {'loss': 2.2178711891174316, 'val_loss': 2.317248821258545}
epoch 6 ended, info: {'loss': 2.2154953479766846, 'val_loss': 2.3151440620422363}
epoch 7 ended, info: {'loss': 2.213690996170044, 'val_loss': 2.313844680786133}
epoch 8 ended, info: {'loss': 2.2122838497161865, 'val_loss': 2.312494993209839}
epoch 9 ended, info: {'loss': 2.210904359817505, 'val_loss': 2.312950849533081}
